In [4]:
%matplotlib inline
%autosave 0
%load_ext autoreload
%autoreload 2

Autosave disabled
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dtrace import rpath, dpath, logger
from dtrace.DTracePlot import DTracePlot
from dtrace.Associations import Association
from dtrace.TargetBenchmark import TargetBenchmark
from dtrace.DTraceEnrichment import DTraceEnrichment

### Import data-sets and associations

In [6]:
assoc = Association(dtype="ic50", load_associations=True, load_ppi=True)

[2019-02-20 17:15:53,361 - INFO]: #(Samples)=500
[2019-02-20 17:15:55,473 - INFO]: #(Drugs)=579; #(Genes)=16818; #(Genomic)=536; 
[2019-02-20 17:16:50,536 - INFO]: ENSP gene map: 19114
[2019-02-20 17:17:30,211 - INFO]: String: 410502
[2019-02-20 17:17:31,765 - INFO]: IGRAPH UN-- 10587 205251 -- 
+ attr: name (v), score (e)
[2019-02-20 17:17:35,309 - INFO]: IGRAPH UN-- 9639 174078 -- 
+ attr: name (v), corr (e), score (e)


In [9]:
assoc.lmm_drug_crispr.head(15)

,DRUG_ID,DRUG_NAME,VERSION,beta,pval,GeneSymbol,n_samples,fdr,DRUG_TARGETS,target_detailed,target
0,1956,MCL1_1284,RS,0.519598,4.222036e-38,MCL1,409,7.100621e-34,MCL1,T,T
1,2354,MCL1_8070,RS,0.519382,4.277158e-37,MCL1,405,7.193324e-33,MCL1,T,T
2,1946,MCL1_5526,RS,0.505459,1.525740e-36,MCL1,409,2.565990e-32,MCL1,T,T
3,2127,Mcl1_6386,RS,0.515574,1.772449e-36,MCL1,407,2.980904e-32,MCL1,T,T
4,1510,Linsitinib,RS,0.514660,2.356474e-35,IGF1R,424,3.963118e-31,IGF1R,T,T
5,2235,AZD5991,RS,0.497077,6.745014e-35,MCL1,406,1.134376e-30,MCL1,T,T
6,1560,Alpelisib,RS,0.491067,9.752904e-35,PIK3CA,423,1.640243e-30,PIK3CA,T,T
7,2125,Mcl1_7350,RS,0.449196,6.905378e-33,MCL1,406,1.161347e-28,MCL1,T,T
8,1114,Cetuximab,v17,0.536082,3.231653e-32,EGFR,443,5.434994e-28,EGFR,T,T
9,1373,Dabrafenib,RS,0.506886,8.120963e-31,BRAF,413,1.365784e-26,BRAF,T,T


In [ ]:
target = TargetBenchmark(assoc=assoc, fdr=0.1)

In [ ]:
# -
e3_ligases = pd.read_excel(f"{dpath}/ubq/Definite Ligase List.xlsx")

In [ ]:
plot_df = assoc.lmm_drug_crispr.query(f"fdr < {target.fdr}")
plot_df["ligase"] = (
    plot_df["GeneSymbol"].isin(e3_ligases["Gene Symbol"]).astype(int).values
)

In [ ]:
background = set(assoc.lmm_drug_crispr["GeneSymbol"])
signature = set(e3_ligases["Gene Symbol"]).intersection(background)
sublist = set(plot_df["GeneSymbol"]).intersection(background)

In [ ]:
pval, int_len = DTraceEnrichment.hypergeom_test(
    signature=signature, background=background, sublist=sublist
)
logger.log(logging.INFO, f"E3 ligases hypergeom: {pval:.2e} (intersection={int_len})")

In [ ]:
# - Non-significant associations description
#
target.signif_essential_heatmap()
plt.gcf().set_size_inches(1, 1)
plt.savefig(
    f"{rpath}/target_benchmark_signif_essential_heatmap.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
#
target.signif_per_screen()
plt.gcf().set_size_inches(0.75, 1.5)
plt.savefig(
    f"{rpath}/target_benchmark_significant_by_screen.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
#
target.signif_genomic_markers()
plt.gcf().set_size_inches(1, 1)
plt.savefig(
    f"{rpath}/target_benchmark_signif_genomic_heatmap.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
#
target.signif_maxconcentration_scatter()
plt.gcf().set_size_inches(2.5, 2.5)
plt.savefig(
    f"{rpath}/target_benchmark_signif_scatter_maxconcentration.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
#
target.signif_fdr_scatter()
plt.gcf().set_size_inches(2, 2)
plt.savefig(
    f"{rpath}/target_benchmark_signif_fdr_scatter.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
# -
target.signif_volcano()
plt.gcf().set_size_inches(1.5, 3)
plt.savefig(
    f"{rpath}/target_benchmark_volcano.pdf", bbox_inches="tight", transparent=True
)

In [ ]:
target.countplot_drugs()
plt.gcf().set_size_inches(2, 0.75)
plt.savefig(
    f"{rpath}/target_benchmark_association_countplot.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
target.countplot_drugs_significant()
plt.gcf().set_size_inches(2, 1)
plt.savefig(
    f"{rpath}/target_benchmark_association_signif_countplot.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
target.pichart_drugs_significant()
plt.gcf().set_size_inches(2, 2)
plt.savefig(
    f"{rpath}/target_benchmark_association_signif_piechart.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
target.boxplot_kinobead()
plt.gcf().set_size_inches(2.5, 0.75)
plt.savefig(
    f"{rpath}/target_benchmark_kinobeads.pdf", bbox_inches="tight", transparent=True
)

In [ ]:
target.beta_histogram()
plt.gcf().set_size_inches(2, 2)
plt.savefig(
    f"{rpath}/target_benchmark_beta_histogram.pdf", bbox_inches="tight", transparent=True
)

In [ ]:
target.pval_histogram()
plt.gcf().set_size_inches(3, 2)
plt.savefig(
    f"{rpath}/target_benchmark_pval_histogram.pdf", bbox_inches="tight", transparent=True
)

In [ ]:
for dtype in ["crispr", "gexp"]:
    target.drugs_ppi(dtype)
    plt.gcf().set_size_inches(2.5, 2.5)
    plt.savefig(
        f"{rpath}/target_benchmark_ppi_distance_{dtype}.pdf",
        bbox_inches="tight",
        transparent=True,
    )

    target.drugs_ppi_countplot(dtype)
    plt.gcf().set_size_inches(2.5, 2.5)
    plt.savefig(
        f"{rpath}/target_benchmark_ppi_distance_{dtype}_countplot.pdf",
        bbox_inches="tight",
        transparent=True,
    )

In [ ]:
target.drugs_ppi_countplot_background()
plt.gcf().set_size_inches(2.5, 2.5)
plt.savefig(
    f"{rpath}/target_benchmark_ppi_distance_{dtype}_countplot_bkg.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
#
target.top_associations_barplot()
plt.savefig(
    f"{rpath}/target_benchmark_associations_barplot.pdf",
    bbox_inches="tight",
    transparent=True,
)

In [ ]:
target.manhattan_plot(n_genes=20)
plt.gcf().set_size_inches(7, 3)
plt.savefig(
    f"{rpath}/drug_associations_manhattan.png",
    bbox_inches="tight",
    transparent=True,
    dpi=600,
)

In [ ]:
# -
drugs = [
    "SN1021632995",
    "AZD5582",
    "SN1043546339",
    "VE-821",
    "AZ20",
    "VE821",
    "VX-970",
    "AZD6738",
    "VE-822",
]

In [ ]:
for d in drugs:
    target.drug_top_associations(d, fdr_thres=0.25)
    plt.gcf().set_size_inches(1.5, 2)
    plt.savefig(
        f"{rpath}/target_benchmark_{d}_top_associations_barplot.pdf",
        bbox_inches="tight",
        transparent=True,
        dpi=600,
    )

In [ ]:
# - Drug targets
genes = ["STAG1", "LIG1", "FLI1", "PARP1", "PARP2", "PARP3", "PCGF5", "XRCC1", "RHNO1"]

In [ ]:
for drug in ["Olaparib", "Talazoparib"]:
    target.drug_notarget_barplot(drug, genes)

    plt.gcf().set_size_inches(2, 1.5)
    plt.savefig(
        f"{rpath}/target_benchmark_drug_notarget_{drug}.pdf",
        bbox_inches="tight",
        transparent=True,
    )

In [ ]:
# - Single feature examples
dgs = [
    ("Alpelisib", "PIK3CA"),
    ("Nutlin-3a (-)", "MDM2"),
    ("MCL1_1284", "MCL1"),
    ("MCL1_1284", "MARCH5"),
    ("Venetoclax", "BCL2"),
    ("AZD4320", "BCL2"),
    ("Volasertib", "PLK1"),
    ("Rigosertib", "PLK1"),
]

In [ ]:
# dg = ('Rigosertib', 'PLK1')
for dg in dgs:
    pair = assoc.lmm_drug_crispr[
        (assoc.lmm_drug_crispr["DRUG_NAME"] == dg[0])
        & (assoc.lmm_drug_crispr["GeneSymbol"] == dg[1])
    ].iloc[0]

    drug = tuple(pair[assoc.dcols])

    dmax = np.log(assoc.drespo_obj.maxconcentration[drug])
    annot_text = f"Beta={pair['beta']:.2g}, FDR={pair['fdr']:.1e}"

    plot_df = pd.concat(
        [
            assoc.drespo.loc[drug].rename("drug"),
            assoc.crispr.loc[dg[1]].rename("crispr"),
            assoc.crispr_obj.institute.rename("Institute"),
        ],
        axis=1,
        sort=False,
    ).dropna()

    g = DTracePlot.plot_corrplot(
        "crispr",
        "drug",
        "Institute",
        plot_df,
        add_hline=False,
        add_vline=False,
        annot_text=annot_text,
    )

    g.ax_joint.axhline(
        y=dmax, linewidth=0.3, color=DTracePlot.PAL_DTRACE[2], ls=":", zorder=0
    )

    g.set_axis_labels(f"{dg[1]} (scaled log2 FC)", f"{dg[0]} (ln IC50)")

    plt.gcf().set_size_inches(1.5, 1.5)
    plt.savefig(
        f"{rpath}/association_drug_scatter_{dg[0]}_{dg[1]}.pdf",
        bbox_inches="tight",
        transparent=True,
    )

In [ ]:
# Drug ~ Gexp
dgs = [
    ("Nutlin-3a (-)", "MDM2"),
    ("Poziotinib", "ERBB2"),
    ("Afatinib", "ERBB2"),
    ("WEHI-539", "BCL2L1"),
]
for dg in dgs:
    pair = assoc.lmm_drug_crispr[
        (assoc.lmm_drug_crispr["DRUG_NAME"] == dg[0])
        & (assoc.lmm_drug_crispr["GeneSymbol"] == dg[1])
    ].iloc[0]

    drug = tuple(pair[assoc.dcols])

    dmax = np.log(assoc.drespo_obj.maxconcentration[drug])
    annot_text = f"Beta={pair['beta']:.2g}, FDR={pair['fdr']:.1e}"

    plot_df = pd.concat(
        [assoc.drespo.loc[drug].rename("drug"), assoc.gexp.loc[dg[1]].rename("crispr")],
        axis=1,
        sort=False,
    ).dropna()
    plot_df["Institute"] = "Sanger"

    g = DTracePlot.plot_corrplot(
        "crispr", "drug", "Institute", plot_df, add_hline=True, annot_text=annot_text
    )

    g.ax_joint.axhline(
        y=dmax, linewidth=0.3, color=DTracePlot.PAL_DTRACE[2], ls=":", zorder=0
    )

    g.set_axis_labels(f"{dg[1]} (voom)", f"{dg[0]} (ln IC50)")

    plt.gcf().set_size_inches(2, 2)
    plt.savefig(
        f"{rpath}/association_drug_gexp_scatter_{dg[0]}_{dg[1]}.pdf",
        bbox_inches="tight",
        transparent=True,
    )

In [ ]:
# CRISPR gene pair corr
for gene_x, gene_y in [("MARCH5", "MCL1"), ("SHC1", "EGFR")]:
    plot_df = pd.concat(
        [
            assoc.crispr.loc[gene_x].rename(gene_x),
            assoc.crispr.loc[gene_y].rename(gene_y),
            assoc.crispr_obj.institute.rename("Institute"),
        ],
        axis=1,
        sort=False,
    ).dropna()

    g = DTracePlot().plot_corrplot(gene_x, gene_y, "Institute", plot_df, add_hline=True)

    g.set_axis_labels(f"{gene_x} (scaled log2 FC)", f"{gene_y} (scaled log2 FC)")

    plt.gcf().set_size_inches(1.5, 1.5)
    plt.savefig(
        f"{rpath}/association_scatter_{gene_x}_{gene_y}.pdf",
        bbox_inches="tight",
        transparent=True,
    )

In [ ]:
# PPI
ppi_examples = [
    ("Nutlin-3a (-)", 0.4, 1, ["RPL37", "UBE3B"]),
    ("AZD3759", 0.3, 1, None),
]
# d, t, o = ('Nutlin-3a (-)', .4, 1, ['RPL37', 'UBE3B'])
for d, t, o, e in ppi_examples:
    graph = assoc.ppi.plot_ppi(
        d,
        assoc.lmm_drug_crispr,
        assoc.ppi,
        corr_thres=t,
        norder=o,
        fdr=0.05,
        exclude_nodes=e,
    )
    graph.write_pdf(f"{rpath}/association_ppi_{d}.pdf")

In [ ]:
# - Betas clustermap
betas_crispr = pd.pivot_table(
    assoc.lmm_drug_crispr.query("VERSION == 'RS'"),
    index=["DRUG_ID", "DRUG_NAME"],
    columns="GeneSymbol",
    values="beta",
)

In [ ]:
#
target.lmm_betas_clustermap(betas_crispr)
plt.gcf().set_size_inches(8, 8)
plt.savefig(
    f"{rpath}/target_benchmark_clustermap_betas_crispr.png", bbox_inches="tight", dpi=300
)

In [ ]:
#
target.lmm_betas_clustermap_legend()
plt.savefig(
    f"{rpath}/target_benchmark_clustermap_betas_crispr_legend.pdf", bbox_inches="tight"
)

In [ ]:
# - Export "all" drug scatter
for d in assoc.drespo.index:
    print(d)
    dassoc = assoc.lmm_drug_crispr[
        (assoc.lmm_drug_crispr["DRUG_NAME"] == d[1])
        & (assoc.lmm_drug_crispr["DRUG_ID"] == d[0])
        & (assoc.lmm_drug_crispr["VERSION"] == d[2])
    ]

    dselected = dassoc.iloc[:5]
    if "T" in dassoc["target"].values:
        dselected = pd.concat([dselected, dassoc[dassoc["target"] == "T"]])

    dmax = np.log(assoc.drespo_obj.maxconcentration[d])

    for i, assoc in dselected.iterrows():
        annot_text = f"Beta={assoc['beta']:.2g}, FDR={assoc['fdr']:.1e}"

        plot_df = pd.concat(
            [
                assoc.drespo.loc[d].rename("drug"),
                assoc.crispr.loc[assoc["GeneSymbol"]].rename("crispr"),
                assoc.crispr_obj.institute.rename("Institute"),
            ],
            axis=1,
            sort=False,
        ).dropna()

        g = DTracePlot.plot_corrplot(
            "crispr",
            "drug",
            "Institute",
            plot_df,
            add_hline=False,
            add_vline=False,
            annot_text=annot_text,
        )

        g.ax_joint.axhline(
            y=dmax, linewidth=0.3, color=DTracePlot.PAL_DTRACE[2], ls=":", zorder=0
        )

        g.set_axis_labels(
            f"{assoc['GeneSymbol']} (scaled log2 FC)",
            f"{d[1]} (ln IC50, {d[0]}, {d[2]})",
        )

        plot_name = f"{rpath}/association_drug_scatters/"
        plot_name += f"{d[1].replace('/', '')} {d[0]} {d[2]} FDR{(assoc['fdr'] * 100):.2f} {assoc['target']} {assoc['GeneSymbol']}.pdf"

        plt.gcf().set_size_inches(1.5, 1.5)
        plt.savefig(plot_name, bbox_inches="tight", transparent=True)